# Terraform (Infrastructure as Code)

## Goals
- Understand Terraform’s core principles: **declarative desired state**, **providers**, **state**, and **plan/apply**.
- Know what Terraform is used for, why teams adopt it, and what it’s capable of.
- See small **pseudo-code** examples of Terraform configuration and workflows.


## Prerequisites
- Basic cloud concepts: networking (VPC/subnets), IAM, regions/zones.
- CLI comfort (running commands, reading diffs).
- Optional: Git + PR workflow for reviewing infrastructure changes.


## What is Terraform?

**Terraform** is an Infrastructure-as-Code (IaC) tool that lets you define infrastructure in configuration files and then **create / update / destroy** real resources via provider APIs (AWS, Azure, GCP, Kubernetes, GitHub, Datadog, etc.).

Terraform configurations describe the **desired end state** (what you want). Terraform computes the set of actions needed to reach that state.


## Core principles (mental model)
- **Declarative configuration**: you describe target resources and their properties; Terraform calculates the change.
- **Providers**: plugins that translate resources into API calls (e.g., `aws_s3_bucket` → AWS S3 API).
- **State**: Terraform records what it previously created in a state file (local or remote). State is how Terraform knows what to change.
- **Plan → Apply safety**: `terraform plan` shows a diff (create/update/delete) before mutating real infrastructure.
- **Dependency graph**: Terraform builds a graph from references to order operations safely.

A useful picture:
`config (HCL) + state  →  plan (diff)  →  apply (API calls)  →  new state`


## What is Terraform used for (practically)?

Terraform is a good fit when you want **repeatable environments** and **reviewable infrastructure changes**.

Common uses:
- Provision cloud infrastructure: networks, compute, storage, databases, IAM.
- Manage platform tooling: Kubernetes clusters/resources, DNS, CDN, observability, CI/CD integrations.
- Standardize architecture patterns via **modules** (reusable building blocks).

Why teams use it:
- **Version control + code review** for infra changes (PRs).
- **Consistency** across dev/stage/prod.
- **Drift detection**: compare desired config vs the real resources.
- **Automation**: CI can run plan/apply with approvals.


## Core concepts (minimum you should know)
- **Resource**: a managed object (e.g., `aws_instance`, `aws_vpc`).
- **Data source**: reads existing info without managing it (e.g., look up an AMI ID).
- **Variables / outputs**: parameterize configs and expose useful results.
- **Modules**: reusable packages of Terraform code (local, Git, registry).
- **Backend**: where state is stored (often remote for teams) + optional **locking**.
- **Workspaces**: separate state instances for the same config (sometimes used for environments).


## Typical workflow
1) `terraform init` (download providers, set up backend)
2) `terraform fmt` / `terraform validate` (format + basic checks)
3) `terraform plan` (see what will change)
4) `terraform apply` (make the change)

In teams, this usually adds:
- Remote state (e.g., S3 backend) + state locking (e.g., DynamoDB).
- CI running `plan` on PRs and `apply` only after approval.


## Example: small AWS stack (pseudo-code)

Below is **illustrative** Terraform-style configuration (HCL). It’s not meant to be copied blindly.

```hcl
# PSEUDO-CODE (HCL)

terraform {
  required_version = ">= 1.6.0"
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }
  }
}

provider "aws" {
  region = var.region
}

variable "region"      { type = string }
variable "bucket_name" { type = string }

resource "aws_s3_bucket" "app" {
  bucket = var.bucket_name
}

resource "aws_s3_bucket_versioning" "app" {
  bucket = aws_s3_bucket.app.id
  versioning_configuration { status = "Enabled" }
}

output "bucket_arn" {
  value = aws_s3_bucket.app.arn
}
```

Typical commands:
```bash
terraform init
terraform plan -out tfplan
terraform apply tfplan
# terraform destroy
```


## What Terraform is capable of
- Provision and manage resources across many providers (cloud + SaaS + on-prem).
- Compose systems from modules (your own or community/registry modules).
- Express dependencies by referencing attributes; Terraform orders operations for you.
- Import existing resources into state (`terraform import`) to bring “manual” infra under management.
- Store state remotely for collaboration and locking.

What it is not:
- Not a general-purpose configuration management tool (tools like Ansible/cloud-init often handle OS-level config).
- Not always ideal when resources are frequently changed out-of-band; drift remediation needs care.


## Pitfalls & quick tips
- Treat state as **sensitive** (it can contain secrets/identifiers). Use remote backends + encryption + access control.
- Pin provider versions and avoid surprise upgrades.
- Prefer `for_each` with stable keys (avoid index-based `count` when ordering can change).
- Use modules to encode best practices (tags, naming, logging, encryption defaults).
- Separate environments cleanly and avoid cross-env coupling via state unless intentional.

## Exercises
- Write a module that creates an S3 bucket with versioning + encryption + public access blocked.
- Add a remote backend configuration (S3 + DynamoDB lock) and explain why locking matters.
- Import an existing bucket with `terraform import` and run `plan` to see drift.

## References
- Terraform Docs: https://developer.hashicorp.com/terraform/docs
- Terraform Language (HCL): https://developer.hashicorp.com/terraform/language
- Providers Registry: https://registry.terraform.io/
